In [17]:
fastas = ["/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Superset/Superset_Thermo_Finetune.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Superset/Superset_Thermo_Test.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Superset/Superset_Test.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Superset/Superset_Train.fa",
          "/home/ubuntu/datasets/alignments/mini_Rfam/mini_rfam.fa",
          "/home/ubuntu/datasets/alignments/5S_rRNA/final_alignments/final_5S_LM.fa",
          "/home/ubuntu/datasets/alignments/16S_rRNA/final_alignments/final_16S_LM.fa", 
          "/home/ubuntu/datasets/alignments/23S_rRNA/final_alignments/final_23S_LM.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Thermophiles/23S_test.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Thermophiles/23S_train.fa",
          "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/23S_train.fa"
         ]
for fa in fastas:
    original_fasta = fa
    out = int(subprocess.check_output(f"seqkit stats {original_fasta}",
                                                        shell=True, text=True).split()[12].replace(",", ""))
    print(fa.split("/")[-1], out)

Superset_Thermo_Finetune.fa 3458844
Superset_Thermo_Test.fa 96379
Superset_Test.fa 30805909
Superset_Train.fa 274214273
mini_rfam.fa 150819705
final_5S_LM.fa 6357623
final_16S_LM.fa 53783103
final_23S_LM.fa 94059751
23S_test.fa 65542
23S_train.fa 1381832
23S_train.fa 88626289


In [20]:
ogt_df = pd.read_csv("OGT_Comparison_Dataset.csv")
thermo_df = ogt_df.loc[ogt_df.TOME_Predicted_OGT_Celsius >= 60]
thermo_acc = thermo_df.GTDB_Genome_Accession.to_list()
fa = "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/Final_Train_Test/Superset/Superset_Train.fa"
np.sum([len(record.seq)for record in SeqIO.parse(fa, "fasta") if record.id.split("__")[0] in thermo_acc])


2605716

In [ ]:
23S Pretrained Thermophilic Tokens: 1,162,403/88,626,289
23S Finetuned Thermophilic Tokens: 1,381,832
231 Pretrained Thermophilic Tokens: 2,605,716/274,214,273
231 Finetuned Thermophilic Tokens: 3,458,844

# <u>SUMMARY</u>

Notebook summarizes hierarchical clustering with CD-HIT-EST and subsequent train/test splits for the following RFAM datasets:
- I. Hierarchical clustering and train/test split of 5S, 16S and 23S datasets
    - Implement individual 95/5 split for number of sequences in 5S, 16S and 23S datasets
- II. Hierarchical Clustering and train/test split: 228 Rfam Set
    - Implement 90/10 split on all tokens in 231 Rfam set
- III. Aggregate 231 Rfam train/test superset:
    - Create supersets for training and testing by concatenating rRNA Rfams with 228 Rfam train/test, repsectively
    - Create thermophile finetuning set:
        - Reserve 10% of all thermophile tokens in superset-test for thermophile-finetuning test set
        - Reserve all thermophile tokens not in the thermophile-finetuning test set for finetuning
  
Code written for AWS Parallel Cluster.

## I. <i>Hierarchical Clustering and train/test split of 5S, 16S and 23S Datasets</i>

In [2]:
#### IMPORT LIBS AND DEF FX's ####
import os, sys, subprocess, glob
import pandas as pd
import numpy as np
import re
from Bio import SeqIO
from collections import defaultdict
from collections import Counter
np.random.seed(31415)

## FUNCTIONS

def subset_fasta_inclusion(id_list_file, sequence_ids, input_fasta, out_fasta):
    """
    Subsets a FASTA file to only include specified sequences.

    Parameters:
    id_list_file (str): File path to save the list of sequence IDs to include.
    sequence_ids (list): List of sequence IDs to include in the subset.
    input_fasta (str): Path to the original FASTA file to subset.
    out_fasta (str): File path to save the subsetted FASTA file.
    """
    print(f"## N_seqs: {len(sequence_ids)}")
    
    # Write the list of IDs to a file
    with open(id_list_file, "w") as f:
        f.writelines("\n".join(sequence_ids))
    
    # Subset the original FASTA file using seqkit
    os.system(f"seqkit grep -f {id_list_file} {input_fasta} > {out_fasta}")

def subset_fasta_exclusion(id_list_file, sequence_ids, input_fasta, out_fasta):
    """
    Subsets a FASTA file to exclude specified sequences.

    Parameters:
    id_list_file (str): File path to save the list of sequence IDs to include.
    sequence_ids (list): List of sequence IDs to include in the subset.
    input_fasta (str): Path to the original FASTA file to subset.
    out_fasta (str): File path to save the subsetted FASTA file.
    """
    print(f"## N_seqs: {len(sequence_ids)}")
    
    # Write the list of IDs to a file
    with open(id_list_file, "w") as f:
        f.writelines("\n".join(sequence_ids))
    
    # Subset the original FASTA file using seqkit
    os.system(f"""seqkit grep -f {id_list_file} -v {input_fasta} > {out_fasta}""")

def get_fasta_ids(input_fasta):
    return [record.id for record in SeqIO.parse(input_fasta, "fasta")]

def write_bash_script(script_id, script_contents):
    """Write bash script from list of commands."""
    with open(script_id, "w") as f:
        f.writelines("\n".join(script_contents))
    return None

def add_suffix_to_fasta(input_file, output_file, suffix):
    """Add suffix (e.g. Rfam ID) to fasta ids"""
    
    # Create seq dict and append Rfam suffix to id
    seq_dict = {f">{record.id}{suffix}":record.seq for record in SeqIO.parse(input_file, "fasta")}
    
    with open(output_file, "w") as output_handle:
        for id_x,seq in seq_dict.items():
            output_handle.write(f"{id_x}\n{seq}\n")

In [2]:
#### CD HIT: Cluster rRNA ####
## VARIABLES 
# Results directory
cluster_dir = "/mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/"

# Customized CD-HIT-EST Executable
cd_hit="/home/ubuntu/software/cdhit_custom/cdhit/cd-hit-est"

# Customized CD-HIT executable for output merging
cluster_rev = "/home/ubuntu/software/cdhit_custom/cdhit/clstr_rev.pl"

# Threads
threads = "$SLURM_CPUS_ON_NODE"

# Rfam rRNA datasets
fasta_dict = {"5S":"/home/ubuntu/datasets/alignments/5S_rRNA/final_alignments/final_5S_LM.fa",
              "16S":"/home/ubuntu/datasets/alignments/16S_rRNA/final_alignments/final_16S_LM.fa", 
              "23S":"/home/ubuntu/datasets/alignments/23S_rRNA/final_alignments/final_23S_LM.fa", 
             }

# Num nodes:
n_nodes = {"5S":1, "16S":1, "23S":2}

## FOR EACH FASTA
bash_scripts = []
all_cluster_dirs = []
for rfam,fasta in fasta_dict.items():
    # SETUP
    out_dir = f"{cluster_dir}{rfam}/"
    all_cluster_dirs.append(out_dir)
    print(rfam,fasta, out_dir)

    # Make subdir for Rfam clusters
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    out_rfam = f"{out_dir}{rfam}_Identity"
    bash_script_name = f"{out_dir}{rfam}_vHierarchical.sh"
    bash_scripts.append(bash_script_name)

    # WRITE BASH SCRIPT
    cd_hit_script = f"""#!/bin/bash
#SBATCH --job-name={rfam}
#SBATCH --output={rfam}job_output.log
#SBATCH --partition=cpu-c6i-16xlarge
#SBATCH --exclusive
#SBATCH --nodes={n_nodes[rfam]}

## SETUP
# ACTIVATE CONDA
source /home/ubuntu/anaconda3/etc/profile.d/conda.sh
conda activate Data_Preprocessing_Env

# VARIABLES
memory=0 # Unlimited = 0  #c6.16xL: 128000 MB
def_line=0 # Grab entire header before first space
mode=1 # Mode 1 has highest accuracy

# START TIME
start_time=$(date +"%Y-%m-%d %H:%M:%S")

## CD-HIT
# CLUSTER HIERARCHICALLY
{cd_hit} -i {fasta} -o {out_rfam}90 -c 0.9  -n 8 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}90 -o {out_rfam}80 -c 0.8  -n 5 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}80 -o {out_rfam}70 -c 0.7  -n 4 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}70 -o {out_rfam}60 -c 0.6  -n 4 -d $def_line -M $memory -g $mode -T {threads}
        
# MERGE
{cluster_rev} {out_rfam}70.clstr {out_rfam}60.clstr > {out_dir}{rfam}_70_60Identity.clstr
{cluster_rev} {out_rfam}80.clstr {out_dir}{rfam}_70_60Identity.clstr > {out_dir}{rfam}_80_70_60Identity.clstr
{cluster_rev} {out_rfam}90.clstr {out_dir}{rfam}_80_70_60Identity.clstr > {out_dir}{rfam}_90_80_70_60Identity.clstr

# END TIME
end_time=$(date +"%Y-%m-%d %H:%M:%S")
# Calculate and print the runtime
start_timestamp=$(date -d "$start_time" +%s)
end_timestamp=$(date -d "$end_time" +%s)
runtime=$((end_timestamp - start_timestamp))
# Print the runtime to the log
echo "Job started at: $start_time"
echo "Job ended at: $end_time"
echo "Total runtime: $runtime seconds"
"""
    # Write Bash script
    write_bash_script(bash_script_name, cd_hit_script)
    
    # Run job
    #os.system(f"sbatch {bash_script_name}")

5S /home/ubuntu/datasets/alignments/5S_rRNA/final_alignments/final_5S_LM.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/5S/
16S /home/ubuntu/datasets/alignments/16S_rRNA/final_alignments/final_16S_LM.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/16S/
23S /home/ubuntu/datasets/alignments/23S_rRNA/final_alignments/final_23S_LM.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/23S/


In [3]:
#### SUBSET FASTAS FOR TRAIN/TEST: rRNA ####
final_train_test_dir = f"{cluster_dir}Final_Train_Test/"

cd_hit_suffix = "_90_80_70_60Identity"
fraction = 0.05 # Fraction of sequences to reserve
seq_margin = 0.01 # Margin of error for sequences to collect
test_size = {}
total_sizes = {}
all_clusters_dict = {}
test_seq_ids = {}
train_fastas = {}

### FOR EACH CLUSTERING RUN, SUBSET THE ORIGINAL FASTA INTO TRAIN/TEST BY RESERVING 5% OF SEQUENCES
# for dir in allall_cluster_dirs:
for dir in [x.replace(cluster_dir, 
                      "/mnt/efs/fs1/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/") for x in all_cluster_dirs]:
    rfam = dir.split("/")[-2]
    print(f"#### PROCESSING: {rfam}")
    out_train = f"{final_train_test_dir}{rfam}_Train.fa"
    train_fastas[rfam] = out_train
    out_test = f"{final_train_test_dir}{rfam}_Test.fa"
    
    # GET CLUSTER FILE
    cluster_file = [x for x in os.listdir(dir) if cd_hit_suffix in x and ".clstr" in x][0]
    with open(f"{dir}{cluster_file}", "r") as f:
        all_clusters = f.read()
    # Split file into dict
    clusters = re.split(r'(>Cluster \d+)', all_clusters)
    cluster_dict = {clusters[i]: [l.split("\t") for l in clusters[i+1].strip().split('\n')] for i in range(1, len(clusters), 2)}
    all_clusters_dict[rfam] = cluster_dict
    
    cluster_dict = dict(sorted(cluster_dict.items(), key=lambda item: len(item[1])))
    sorted_keys = list(cluster_dict.keys())

    # ORIGINAL FASTA
    original_fasta = fasta_dict[rfam]
    num_original_entries = int(subprocess.check_output(["grep", "-c", ">", original_fasta]).decode("utf-8").strip())
    total_sizes[rfam] = num_original_entries
    max_ids = int(np.ceil(num_original_entries*fraction)) + int(np.ceil(num_original_entries*seq_margin))
    min_ids = int(np.ceil(num_original_entries*fraction)) - int(np.ceil(num_original_entries*seq_margin))
    print(f"Min_Ids: {min_ids}, Max_Ids: {max_ids}")
    
    ## SUBSET DATA
    cluster_members = []
    key_counter = 0
    while max_ids > len(cluster_members):
        key = sorted_keys[key_counter]
        cluster_x_members = [re.split(r'>(.*)\.\.\.',s[1])[1] for s in cluster_dict[key]]
        print(f"# PROCESSING {key}: {len(cluster_x_members)} Members.")
        key_counter += 1
        
        if (len(cluster_x_members)+len(cluster_members) > max_ids and len(cluster_x_members)+len(cluster_members) >= min_ids):
            print(f"! EXLCUDING Cluster {key}, of size {len(cluster_x_members)} to avoid exceeding Max ({max_ids}) for current set (size = {len(cluster_members)}).")
            break # If min_ids satisfied, break loop before exceeding maxids 
        
        cluster_members += cluster_x_members

    test_size[rfam] = len(cluster_members)
    test_seq_ids[rfam] = cluster_members
    test_id_list_file = f"{final_train_test_dir}{rfam}_ID_List.txt" # List of IDs, required for filtering
    
    # TRAIN
    if not os.path.exists(out_train):
        subset_fasta_exclusion(test_id_list_file, cluster_members, original_fasta, out_train)
    
    # TEST
    if not os.path.exists(out_test):
        subset_fasta_inclusion(test_id_list_file, cluster_members, original_fasta, out_test)
        os.remove(test_id_list_file) 
        

#### PROCESSING: 5S
Min_Ids: 2204, Max_Ids: 3306
# PROCESSING >Cluster 12: 2 Members.
# PROCESSING >Cluster 11: 3 Members.
# PROCESSING >Cluster 9: 7 Members.
# PROCESSING >Cluster 4: 93 Members.
# PROCESSING >Cluster 6: 158 Members.
# PROCESSING >Cluster 7: 214 Members.
# PROCESSING >Cluster 3: 721 Members.
# PROCESSING >Cluster 10: 855 Members.
# PROCESSING >Cluster 8: 1587 Members.
! EXLCUDING Cluster >Cluster 8, of size 1587 to avoid exceeding Max (3306) for current set (size = 2053).
## N_seqs: 2053


[INFO] 2053 patterns loaded from file


## N_seqs: 2053
#### PROCESSING: 16S
Min_Ids: 1422, Max_Ids: 2134
# PROCESSING >Cluster 14: 1 Members.
# PROCESSING >Cluster 9: 2 Members.
# PROCESSING >Cluster 13: 3 Members.
# PROCESSING >Cluster 5: 4 Members.
# PROCESSING >Cluster 12: 8 Members.
# PROCESSING >Cluster 6: 9 Members.
# PROCESSING >Cluster 4: 10 Members.
# PROCESSING >Cluster 3: 30 Members.
# PROCESSING >Cluster 10: 64 Members.
# PROCESSING >Cluster 8: 110 Members.
# PROCESSING >Cluster 7: 119 Members.
# PROCESSING >Cluster 11: 658 Members.
# PROCESSING >Cluster 2: 677 Members.
# PROCESSING >Cluster 1: 6916 Members.
! EXLCUDING Cluster >Cluster 1, of size 6916 to avoid exceeding Max (2134) for current set (size = 1695).
## N_seqs: 1695


[INFO] 2053 patterns loaded from file
[INFO] 1695 patterns loaded from file


## N_seqs: 1695
#### PROCESSING: 23S
Min_Ids: 1292, Max_Ids: 1940
# PROCESSING >Cluster 21: 1 Members.
# PROCESSING >Cluster 25: 1 Members.
# PROCESSING >Cluster 31: 1 Members.
# PROCESSING >Cluster 45: 1 Members.
# PROCESSING >Cluster 52: 1 Members.
# PROCESSING >Cluster 54: 1 Members.
# PROCESSING >Cluster 57: 1 Members.
# PROCESSING >Cluster 61: 1 Members.
# PROCESSING >Cluster 63: 1 Members.
# PROCESSING >Cluster 64: 1 Members.
# PROCESSING >Cluster 65: 1 Members.
# PROCESSING >Cluster 66: 1 Members.
# PROCESSING >Cluster 68: 1 Members.
# PROCESSING >Cluster 73: 1 Members.
# PROCESSING >Cluster 75: 1 Members.
# PROCESSING >Cluster 77: 1 Members.
# PROCESSING >Cluster 46: 2 Members.
# PROCESSING >Cluster 51: 2 Members.
# PROCESSING >Cluster 58: 2 Members.
# PROCESSING >Cluster 67: 2 Members.
# PROCESSING >Cluster 78: 2 Members.
# PROCESSING >Cluster 79: 2 Members.
# PROCESSING >Cluster 81: 2 Members.
# PROCESSING >Cluster 4: 3 Members.
# PROCESSING >Cluster 26: 3 Members.
# PROCESSI

[INFO] 1695 patterns loaded from file
[INFO] 1831 patterns loaded from file


## N_seqs: 1831


[INFO] 1831 patterns loaded from file


In [4]:
#### CREATE THERMOPHILE FINETUNING SET: finetune on all thermophile rRNA seqs, but finetune test cannot contain seqs in original train ####
## GET THERMOPHILE IDs
ogt_df = pd.read_csv("OGT_Comparison_Dataset.csv")
thermo_df = ogt_df.loc[ogt_df.TOME_Predicted_OGT_Celsius >= 60]
thermo_acc = thermo_df.GTDB_Genome_Accession.to_list()
max_test_seqs = 0.10

## CREATE FINETUNE AND THERMO-TEST
for rfam,original_fasta in fasta_dict.items():
    # Original fasta info
    original_thermo_ids = [x for x in get_fasta_ids(original_fasta) if x in thermo_acc]
    print(f"### {rfam} Total Thermo: {len(original_thermo_ids)}")
    # Training set info
    train_fasta = train_fastas[rfam]
    train_thermo_ids = [x for x in get_fasta_ids(train_fasta) if x in thermo_acc]

    # Thermo test ids: reserve all thermos not in the initial training set for thermo-testing
    thermo_test_ids = [x for i,x in enumerate(original_thermo_ids) if x not in train_thermo_ids \
                       and i/len(original_thermo_ids)<= max_test_seqs]
    print(f"# {rfam} Thermo-Test Ids: {len(thermo_test_ids)}")

    # Thermo finetuning ids: Use all thermophile not reserved for thermo-test
    finetune_ids = [x for x in original_thermo_ids if x not in thermo_test_ids]
    
    # Create thermo test set
    test_id_list_file = f"{final_train_test_dir}{rfam}_THERMO_Test_Ids.txt"
    thermo_test_fasta = f"{final_train_test_dir}{rfam}_THERMO_Test.fa"
    if not os.path.exists(thermo_test_fasta):
        subset_fasta_inclusion(test_id_list_file, thermo_test_ids, original_fasta, thermo_test_fasta)
            
    # Create thermo Finetuning set
    finetune_fasta = f"{final_train_test_dir}{rfam}_THERMO_Finetune.fa"
    if not os.path.exists(out_test):
        subset_fasta_inclusion(test_id_list_file, finetune_ids, original_fasta, finetune_fasta)
        # Clean up
        if os.path.exists(test_id_list_file):
            os.remove(test_id_list_file)
            

### 5S Total Thermo: 931
# 5S Thermo-Test Ids: 0
## N_seqs: 0


[WARN] 0 patterns loaded from file


### 16S Total Thermo: 540
# 16S Thermo-Test Ids: 54
## N_seqs: 54


[INFO] 54 patterns loaded from file


### 23S Total Thermo: 482
# 23S Thermo-Test Ids: 9
## N_seqs: 9


[INFO] 9 patterns loaded from file


## II. Hierarchical Clustering and train/test split: 228 Rfam Set

In [5]:
#### CD HIT: CLUSTER ALL ####
## VARIABLES
# Data
rfam_228_dir = "/home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/"
fasta_dict_228 = {fa.split("/")[-1].split("_final.fa")[0]:fa for fa in glob.glob(f"{rfam_228_dir}*_final.fa")}
rfam_3x_test = {"5S" : f"{final_train_test_dir}5S_Test.fa", 
                "16S" : f"{final_train_test_dir}16S_Test.fa", 
                "23S" : f"{final_train_test_dir}23S_Test.fa"}
rfam_3x_train = {"5S": f"{final_train_test_dir}5S_Train.fa",
                "16S": f"{final_train_test_dir}16S_Train.fa",
                "23S": f"{final_train_test_dir}23S_Train.fa"}
# ADD RFAM SUFFIX TO ORIGINAL FASTA HEADERS
new_suffix = "_with_Rfam_suffix.fa"
for rfam,fasta in fasta_dict.items():
    new_fasta = f"{final_train_test_dir}{rfam}_Total{new_suffix}"
    if not os.path.exists(new_fasta):
        add_suffix_to_fasta(fasta, new_fasta, f"__{rfam}")
        fasta_dict[rfam] = new_fasta
for rfam,fasta in rfam_3x_test.items():
    new_fasta = fasta.replace(".fa", new_suffix)
    if not os.path.exists(new_fasta):
        add_suffix_to_fasta(fasta, new_fasta, f"__{rfam}")
        rfam_3x_test[rfam] = new_fasta
for rfam,fasta in rfam_3x_train.items():
    new_fasta = fasta.replace(".fa", f"__{rfam}")
    if not os.path.exists(new_fasta):
        add_suffix_to_fasta(fasta, new_fasta, f"__{rfam}")
        rfam_3x_train[rfam] = new_fasta

## FOR EACH FASTA
bash_scripts = []
for rfam,fasta in fasta_dict_228.items():
    # SETUP
    out_dir = f"{cluster_dir}{rfam}/"
    print(rfam,fasta, out_dir)
    
    # Make subdir for Rfam clusters
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    out_rfam = f"{out_dir}{rfam}_Identity"
    bash_script_name = f"{out_dir}{rfam}_vHierarchical.sh"
    bash_scripts.append(bash_script_name)
    cd_hit_script = f"""#!/bin/bash
#SBATCH --job-name={rfam}
#SBATCH --output={rfam}job_output.log
#SBATCH --partition=cpu-c6i-16xlarge
#SBATCH --exclusive
#SBATCH --nodes=1

## SETUP
# ACTIVATE CONDA
source /home/ubuntu/anaconda3/etc/profile.d/conda.sh
conda activate bioinfo_tools_Py_310

# VARIABLES
memory=0 #Unlimited = 0  #c6.16xL: 128000 MB
def_line=0 #Grab entire header before first space
mode=1 #1 is accurate mode

# START TIME
start_time=$(date +"%Y-%m-%d %H:%M:%S")

## CD-HIT
# CLUSTER HIERARCHICALLY
{cd_hit} -i {fasta} -o {out_rfam}90 -c 0.9  -n 8 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}90 -o {out_rfam}80 -c 0.8  -n 5 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}80 -o {out_rfam}70 -c 0.7  -n 4 -d $def_line -M $memory -g $mode -T {threads}
{cd_hit} -i {out_rfam}70 -o {out_rfam}60 -c 0.6  -n 4 -d $def_line -M $memory -g $mode -T {threads}
        
#MERGE
{cluster_rev} {out_rfam}70.clstr {out_rfam}60.clstr > {out_dir}{rfam}_70_60Identity.clstr
{cluster_rev} {out_rfam}80.clstr {out_dir}{rfam}_70_60Identity.clstr > {out_dir}{rfam}_80_70_60Identity.clstr
{cluster_rev} {out_rfam}90.clstr {out_dir}{rfam}_80_70_60Identity.clstr > {out_dir}{rfam}_90_80_70_60Identity.clstr

# END TIME
end_time=$(date +"%Y-%m-%d %H:%M:%S")
# Calculate and print the runtime
start_timestamp=$(date -d "$start_time" +%s)
end_timestamp=$(date -d "$end_time" +%s)
runtime=$((end_timestamp - start_timestamp))
# Print the runtime to the log
echo "Job started at: $start_time"
echo "Job ended at: $end_time"
echo "Total runtime: $runtime seconds"
"""
    # Write Bash script
    write_bash_script(bash_script_name, cd_hit_script)
    
    # Run job
    #os.system(f"sbatch {bash_script_name}")


RF01702 /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF01702_final.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/RF01702/
RF02431 /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02431_final.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/RF02431/
RF02035 /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02035_final.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/RF02035/
RF02969 /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02969_final.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/RF02969/
RF01720 /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF01720_final.fa /mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/RF01720/
RF00514 /home/ubuntu/datasets/alignments

In [6]:
#### GET FASTA STATS ###
## CHECK AVAILABLE RFAMS
old_rfams = ["5S", "16S", "23S"]
new_rfams = [x for x in  os.listdir(cluster_dir) if x.startswith("RF")]
print(f"#### Num RFAMS in miniRFAM: {len(new_rfams)}")

## SEQ LENS:
minirfam_fasta_len_dict = {record.id:len(record.seq) for record in \
                           SeqIO.parse("/home/ubuntu/datasets/alignments/mini_Rfam/mini_rfam.fa", "fasta")}

## NEW FASTAS: create dict of individual fasta splits
suffix_228_fa = "_final.fa"

## COUNT TOTAL FASTAS: Count seqs/rfam and tokens/rfam
bp_per_rfam = {}
n_seqs_per_rfam = {}

thermo_bp_per_rfam = {}
thermo_n_seqs_per_rfam = {}

for k,v in fasta_dict.items():
    output = subprocess.check_output(f"seqkit stats {v}", shell=True, text=True)
    # Total Tokens or BP
    bp_per_rfam[k] = int(output.split()[12].replace(",", ""))
    # Total Seqs
    n_seqs_per_rfam[k] = int(output.split()[11].replace(",", ""))
for k,v in fasta_dict_228.items():    
    output = subprocess.check_output(f"seqkit stats {v}", shell=True, text=True)
    # Total Tokens or BP
    bp_per_rfam[k] = int(output.split()[12].replace(",", ""))
    # Total Seqs
    n_seqs_per_rfam[k] = int(output.split()[11].replace(",", ""))

# FIND RFAMS WITHOUT THERMOs
no_thermophiles_in_rfam = [k for k,v in thermo_n_seqs_per_rfam.items() if v == 0 ] #and k not in old_rfams
# FIND RFAMS WITH THERMOs
rfams_with_thermos = [k for k,v in thermo_n_seqs_per_rfam.items() if v != 0 ] # and k not in old_rfams

# GET TOTAL TOKENS: Find 5S, 16S, 23S token counts 
total_token_3xrfam = 0
for rfam,fasta in fasta_dict.items():
    output = subprocess.check_output(f"seqkit stats {fasta.replace('Superset', 'Superset/Archive')}", 
                                     shell=True, text=True)
    test_tokens = int(output.split()[12].replace(",", ""))
    total_token_3xrfam += test_tokens
    print(rfam,total_token_3xrfam)
    
total_tokens_228xrfam = int(subprocess.check_output(f"seqkit stats /home/ubuntu/datasets/alignments/mini_Rfam/mini_rfam.fa", 
                                     shell=True, text=True).split()[12].replace(",",""))
total_tokens_231xrfam = total_tokens_228xrfam + total_token_3xrfam
print(total_tokens_231xrfam)


#### Num RFAMS in miniRFAM: 228
5S 6357623
16S 60140726
23S 154200477
305020182


In [7]:
#### COUNT CLUSTER STATS ####
## ALL DATA
cd_hit_suffix = "_90_80_70_60Identity"
print("#### ALL DATA ####")
all_clusters_dict = {}
cluster_count_dict = {}
seqs_per_cluster_dict = {}
tokens_per_cluster_dict = {}
tokens_per_cluster_from_thermos_dict = {}
cluster_to_seqid_list_dicts = {}
clusters_without_thermos_dict = {}
clusters_with_thermos_dict = {}
seq_ids_to_cluster_dict_of_dicts = {}
singlets = []

# FOR FASTA IN MINI RFAMs, PARSE CLUSTER INFO
for rfam,fasta in fasta_dict_228.items():
    dir = "/home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/"+f"{rfam}/" #f"{cluster_dir}{rfam}/"
    print(f"#### PROCESSING: {rfam}, {dir}")
    
    # GET CLUSTER FILE
    cluster_file = [x for x in os.listdir(dir) if cd_hit_suffix in x and ".clstr" in x][0]
    with open(f"{dir}{cluster_file}", "r") as f:
        all_clusters = f.read()
    # Split file into dict
    clusters = re.split(r'(>Cluster \d+)', all_clusters)
    cluster_dict = {clusters[i]: [l.split("\t") for l in clusters[i+1].strip().split('\n')] for i in range(1, len(clusters), 2)}
    all_clusters_dict[rfam] = cluster_dict

    cluster_to_seqid_list_dict = {key: [re.split(r'>(.*)\.\.\.', s[1])[1] for s in cluster_dict[key]] for key in cluster_dict}
    cluster_to_seqid_list_dicts[rfam] = {key: [re.split(r'>(.*)\.\.\.', s[1])[1] for s in cluster_dict[key]] for key in cluster_dict}
    
    seq_ids_to_cluster_dict = {item: key for key, items_list in cluster_to_seqid_list_dict.items() for item in items_list}
    seq_ids_to_cluster_dict_of_dicts[rfam] = seq_ids_to_cluster_dict
    
    # COUNT CLUSTERS
    n_clusters = len(cluster_dict.keys())
    cluster_count_dict[rfam] = n_clusters
    if n_clusters == 1:
        singlets.append(rfam)
    
    # SEQS PER CLUSTER
    seqs_per_cluster = {len(v) for k,v in cluster_dict.items()}
    seqs_per_cluster_dict[rfam] = seqs_per_cluster

    # COUNT TOKEN COUNTS FOR GENERAL AND THERMOs
    cluster_seq_len_dict = defaultdict(dict)
    cluster_THERMO_seq_len_dict = defaultdict(dict)
    
    for record in SeqIO.parse(fasta, "fasta"):
        # Get and clean fasta id
        seq_id = record.id#.split("__")[1]
        gtdb_id = record.id.split("__")[0]
        seq_len = len(record.seq)
        #cluster_id = seq_ids_to_cluster_dict[seq_id.split("__")[1]]
        cluster_id = seq_ids_to_cluster_dict[seq_id]
        # Save length of seq in general nested dict
        cluster_seq_len_dict[cluster_id][seq_id] = seq_len
        # Save length of seq in thermo nested dict
        if gtdb_id in thermo_acc:
            cluster_THERMO_seq_len_dict[cluster_id][seq_id] = seq_len
    
    # Collapse data as cluster sums
    tokens_per_cluster = {k:np.sum(list(v.values())) for k,v in cluster_seq_len_dict.items()}
    tokens_per_cluster_from_thermos = {k:np.sum(list(v.values())) for k,v in cluster_THERMO_seq_len_dict.items()}
    clusters_without_thermos = [x for x in tokens_per_cluster.keys() if x not in tokens_per_cluster_from_thermos.keys()]
    clusters_with_thermos = [x for x in tokens_per_cluster.keys() if x in tokens_per_cluster_from_thermos.keys()]
    # Save
    tokens_per_cluster_dict[rfam] = tokens_per_cluster
    tokens_per_cluster_from_thermos_dict[rfam] = tokens_per_cluster_from_thermos
    clusters_without_thermos_dict[rfam] = clusters_without_thermos
    clusters_with_thermos_dict[rfam] = clusters_with_thermos

    print(f"#### {rfam}: {n_seqs_per_rfam[rfam]} sequences in RFAM, "
          f"{n_clusters} clusters, {bp_per_rfam[rfam]} total tokens, {len(clusters_with_thermos)} thermo clusters" )
all_data_thermo_seq_ids_per_cluster_dict = {
    k: {
        cluster_id: [x for x in seqids if x.split("__")[0] in thermo_acc]
        for cluster_id, seqids in v.items()
        if len([x for x in seqids if x.split("__")[0] in thermo_acc]) != 0  # Filter out empty values
    }
    for k, v in cluster_to_seqid_list_dicts.items()
}    


#### ALL DATA ####
#### PROCESSING: RF01702, /home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/RF01702/
#### RF01702: 53 sequences in RFAM, 2 clusters, 10837 total tokens, 0 thermo clusters
#### PROCESSING: RF02431, /home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/RF02431/
#### RF02431: 204 sequences in RFAM, 1 clusters, 29381 total tokens, 0 thermo clusters
#### PROCESSING: RF02035, /home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/RF02035/
#### RF02035: 583 sequences in RFAM, 1 clusters, 82205 total tokens, 0 thermo clusters
#### PROCESSING: RF02969, /home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/RF02969/
#### RF02969: 212 sequences in RFAM, 8 clusters, 24189 total tokens, 0 thermo clusters
#### PROCESSING: RF01720, /home/ubuntu/datasets/db1/traintestval_split/CD_Hit/Custom_CD_Hit/Hierarchical_Clustering/RF01720/
#### RF01

In [8]:
#### TRAIN/TEST SPLITS: Total 228x Dataset ####
# GET TOTAL TOKENS IN 3xRFAM TEST
total_token_test_3xrfam = 0
for rfam,fasta in rfam_3x_test.items():
    output = subprocess.check_output(f"seqkit stats {fasta.replace('Superset', 'Superset/Archive')}", 
                                     shell=True, text=True)
    test_tokens = int(output.split()[12].replace(",", ""))
    total_token_test_3xrfam += test_tokens
    print(rfam,test_tokens)
print(f"# 3x RFAM TEST TOKENS: {total_token_test_3xrfam}")

# COLLECT 10% TOTAL TOKENS FOR 231 RFAMs
fraction = 0.10
seq_margin = 0.01
ten_percent_tokens = int(np.ceil(fraction*total_tokens_231xrfam))
min_test_tokens = ten_percent_tokens - int(np.ceil(fraction*total_tokens_231xrfam*seq_margin))
max_test_tokens = ten_percent_tokens + int(np.ceil(fraction*total_tokens_231xrfam*seq_margin))
rfams_processed_for_test = []
test_set_ids = []
train_fastas = []
test_fastas = []
test_tokens_per_rfam = {}
current_tokens = 0

## PROCESS NON-SINGLET RFAMS
# FIND RFAMS WITH >1 CLUSTER
not_singlets = [x for x in all_clusters_dict.keys() if x not in singlets]

# FIND RFAMS WITH MODEST CLUSTER SIZES: get clusters that aren't >33% RFam's tokens
dominant_fraction = 0.33 # Define dominant cluster
not_singlet_with_modest_token_clusters = {} # Dict of dicts: major_key =  RFAM , minor_key = cluster_ids, minor_vals = cluster_token_count
for rfam in not_singlets:
    cluster_token_dict = tokens_per_cluster_dict[rfam]
    modest_clusters = {}
    percent_x = dominant_fraction*np.sum(list(cluster_token_dict.values()))
    # Find Clusters that aren't > 33% Rfam's tokens
    for cid,token_count in cluster_token_dict.items():
        if token_count > percent_x:
            continue
        else:
            modest_clusters[cid] = token_count
    # Save rfams with modest clusters
    if len(modest_clusters.values()) == 0: # Skip if rfam only had large clusters
        continue
    else:
        not_singlet_with_modest_token_clusters[rfam] = modest_clusters

# COLLECT < 33% TOKENS FROM NON-SINGLET RFAMS WITH MODEST CLUSTER SETS
rfam_fraction = dominant_fraction
modest_counter = 0 

while current_tokens < max_test_tokens:
    if modest_counter >= len(not_singlet_with_modest_token_clusters.keys()): # Prevent infinite loop
        break
    for rfam,good_clusters in not_singlet_with_modest_token_clusters.items():
        if rfam in rfams_processed_for_test:
            continue
        
        original_fasta = fasta_dict_228[rfam]
        print(f"#### PROCESSING Non-singlet modest-cluster RFAM: {rfam}, {original_fasta}")
        out_train = f"{final_train_test_dir}{rfam}_train.fa"
        out_test = f"{final_train_test_dir}{rfam}_test.fa"
        
        modest_cluster_ids = list(good_clusters.keys())
        n_modest_clusters = len(modest_cluster_ids)
        
        total_token_count = int(subprocess.check_output(f"seqkit stats {original_fasta}",
                                                        shell=True, text=True).split()[12].replace(",", ""))
        rfam_x_max_tokens = int(np.ceil(total_token_count*rfam_fraction)) + int(np.ceil(total_token_count*rfam_fraction)) * seq_margin

        # Iterate through modest clusters
        temp_tokens = 0
        cluster_counter = 0
        sequence_ids = [] # Selected clusters
        while temp_tokens < rfam_x_max_tokens:
            # Randomly select modest cluster
            selected = []
            random_cluster_id = np.random.choice(modest_cluster_ids)
            while random_cluster_id in selected:
                random_cluster_id = np.random.choice(modest_cluster_ids)
            if random_cluster_id not in selected:
                selected.append(random_cluster_id)
            
            # Get num tokens in random choice
            random_cluster_seq_ids = list(set(cluster_to_seqid_list_dicts[rfam][random_cluster_id]))
            tokens_in_random_cluster = np.sum([minirfam_fasta_len_dict[x] for x in random_cluster_seq_ids])
            #tokens_in_random_cluster = good_clusters[random_cluster_id]
            
            # Assess cluster's token count and add to sequence_ids if within min/max boundaries
            if (temp_tokens+tokens_in_random_cluster <= rfam_x_max_tokens) and (cluster_counter <= n_modest_clusters)\
                and ((current_tokens + temp_tokens) < max_test_tokens):
                sequence_ids.append(random_cluster_seq_ids)
                temp_tokens += tokens_in_random_cluster
            elif cluster_counter > n_modest_clusters:
                break # break if you've iterated through all clusters
            cluster_counter += 1
        
        # UPDATE TOKEN COUNT
        if (current_tokens + temp_tokens) > max_test_tokens:
            modest_counter += 1
            print(f"~~~~Skipping: {rfam}")
            continue
        else:
            if rfam in rfams_processed_for_test:
                continue
            test_tokens_per_rfam[rfam] = temp_tokens
            
            # WRITE TRAIN/TEST
            # Format ID list
            sequence_ids = list(set([s for l in sequence_ids for s in l]))  # Flatten
            test_set_ids.append(sequence_ids)
            id_list_file = f"{final_train_test_dir}{rfam}_ID_List.txt"
            print(f"## N_seqs: {len(sequence_ids)}")
            
            # Test
            if not os.path.exists(out_test):
                subset_fasta_inclusion(id_list_file, sequence_ids, original_fasta, out_test)
            
            # Train
            if not os.path.exists(out_train):
                subset_fasta_exclusion(id_list_file, sequence_ids, original_fasta, out_train)
            
            train_fastas.append(out_train)
            test_fastas.append(out_test)
            
            # Clean Dir
            os.remove(id_list_file)
            
            rfams_processed_for_test.append(rfam)
            modest_counter += 1

            # Update token counter
            flat_test_ids = [s for l in test_set_ids for s in l]
            current_tokens = np.sum([minirfam_fasta_len_dict[x] for x in flat_test_ids]) + total_token_test_3xrfam
print((f"!!!! TOKENS BEFORE SINGLETS: {current_tokens}"))


## PROCESS SINGLET RFAMS
# PROCESS REMAINING SINGLETS
singlet_counter = 0
included_singlet_counter = []
unprocessed_singlets = [x for x in singlets if x not in rfams_processed_for_test] # Skip previously processed rfams
max_singlet_iters =  len(unprocessed_singlets)

while current_tokens < max_test_tokens:
    selected_rfams = []
    # Randomly select singlet-rfam
    rfam = np.random.choice(unprocessed_singlets)
    while rfam in selected_rfams:
        rfam = np.random.choice(unprocessed_singlets)
        if len(rfams_processed_for_test) > len(fasta_dict_228.keys()):
            sys.exit()
    if rfam in rfams_processed_for_test:
        continue
    else:
        selected_rfams.append(rfam)
        
    # Prevent infinite loop: break if you've iterated through all singlets and don't have min token count
    if singlet_counter > max_singlet_iters:
        sys.exit()
        
    original_fasta = fasta_dict_228[rfam]
    print(f"#### PROCESSING SINGLET RFAM: {rfam}, {original_fasta}")
    out_train = f"{final_train_test_dir}{rfam}_train.fa"
    out_test = f"{final_train_test_dir}{rfam}_test.fa"

    # COLLECT TOKENS AND SEQ IDS
    # Get token count for rfam
    temp_tokens = int(subprocess.check_output(f"seqkit stats {fasta_dict_228[rfam]}", 
                      shell=True, text=True).split()[12].replace(",", "")) #bp_per_rfam[rfam] #list(tokens_per_cluster_dict[rfam].values())[0]
    print(rfam, temp_tokens)
    
    # Get Seq IDs
    sequence_ids = list(cluster_to_seqid_list_dicts[rfam].values())[0]
    
    # UPDATE TOKEN COUNT
    if (current_tokens + temp_tokens) > max_test_tokens:
        singlet_counter +=1
        print(f"~~~~Skipping: {rfam}")
        continue # Skip if singlet token count too large
    else:
        if rfam in rfams_processed_for_test:
            continue
        test_tokens_per_rfam[rfam] = temp_tokens    
        included_singlet_counter.append(rfam)
        # WRITE TRAIN/TEST
        # Write ID list
        test_set_ids.append(sequence_ids)
        id_list_file = f"{final_train_test_dir}{rfam}_ID_List.txt"
        print(f"## N_seqs: {len(sequence_ids)}")

        # Test
        if not os.path.exists(out_test):
            subset_fasta_inclusion(id_list_file, sequence_ids, original_fasta, out_test)
        # Train
        if not os.path.exists(out_train):
            subset_fasta_exclusion(id_list_file, sequence_ids, original_fasta, out_train)        
        
        train_fastas.append(out_train)
        test_fastas.append(out_test)
            
        # Clean Dir
        os.remove(id_list_file)
        rfams_processed_for_test.append(rfam)
        
        # Update token counter
        singlet_counter +=1
        flat_test_ids = [s for l in test_set_ids for s in l]
        current_tokens = np.sum([minirfam_fasta_len_dict[x] for x in flat_test_ids]) + total_token_test_3xrfam

5S 234695
16S 2507382
23S 5433462
# 3x RFAM TEST TOKENS: 8175539
#### PROCESSING Non-singlet modest-cluster RFAM: RF02969, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02969_final.fa
## N_seqs: 40
## N_seqs: 40
## N_seqs: 40


[INFO] 40 patterns loaded from file
[INFO] 40 patterns loaded from file
[INFO] 275 patterns loaded from file


#### PROCESSING Non-singlet modest-cluster RFAM: RF00514, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00514_final.fa
## N_seqs: 275
## N_seqs: 275
## N_seqs: 275
#### PROCESSING Non-singlet modest-cluster RFAM: RF01405, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF01405_final.fa
## N_seqs: 101
## N_seqs: 101


[INFO] 275 patterns loaded from file
[INFO] 101 patterns loaded from file
[INFO] 101 patterns loaded from file


## N_seqs: 101
#### PROCESSING Non-singlet modest-cluster RFAM: RF00028, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00028_final.fa
## N_seqs: 12
## N_seqs: 12
## N_seqs: 12


...

#### PROCESSING Non-singlet modest-cluster RFAM: RF00162, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00162_final.fa
~~~~Skipping: RF00162
#### PROCESSING Non-singlet modest-cluster RFAM: RF02502, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02502_final.fa
~~~~Skipping: RF02502
!!!! TOKENS BEFORE SINGLETS: 30800847
#### PROCESSING SINGLET RFAM: RF00166, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00166_final.fa
RF00166 106411
~~~~Skipping: RF00166
#### PROCESSING SINGLET RFAM: RF00111, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00111_final.fa
RF00111 22889
~~~~Skipping: RF00111
#### PROCESSING SINGLET RFAM: RF00444, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00444_final.fa
RF00444 221531
~~~~Skipping: RF00444
#### PROCESSING SINGLET RFAM: RF02422, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02422_final.fa
RF02422 50199
~~~~Skipping: RF02422
#### PROCESSIN

[INFO] 10 patterns loaded from file
[INFO] 10 patterns loaded from file


## N_seqs: 10
#### PROCESSING SINGLET RFAM: RF00113, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00113_final.fa
RF00113 24816
~~~~Skipping: RF00113
#### PROCESSING SINGLET RFAM: RF02355, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02355_final.fa
RF02355 70401
~~~~Skipping: RF02355
#### PROCESSING SINGLET RFAM: RF02680, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02680_final.fa
RF02680 9733
~~~~Skipping: RF02680
#### PROCESSING SINGLET RFAM: RF00378, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF00378_final.fa
RF00378 66178
~~~~Skipping: RF00378
#### PROCESSING SINGLET RFAM: RF02345, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02345_final.fa
RF02345 136783
~~~~Skipping: RF02345
#### PROCESSING SINGLET RFAM: RF02235, /home/ubuntu/datasets/alignments/mini_Rfam/final_split_by_model/RF02235_final.fa
RF02235 39492
~~~~Skipping: RF02235
#### PROCESSING SINGLET RFAM: RF02356, /home/ubunt

SystemExit: 

/home/ubuntu/anaconda3/envs/bioinfo_tools_Py_310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## III. Aggregate 231 Rfam train/test superset

In [9]:
#### CONCATENATE SUPERSET AND STATS ####
superset_dir = "/mnt/efs/fs1/software/Manuscript/GTDB_Data_Curation_and_Preprocessing/Hierarchical_Clustering/Final_Train_Test/Superset/"
total_228x_fasta = "/home/ubuntu/datasets/alignments/mini_Rfam/mini_rfam.fa"

## COLLECT SUPERSET TRAIN: 3xRFAM Train + 228XRFAM Train
# Add old rfams and rfams with dominant clusters
unprocessed_train_fastas = [fasta for rfam,fasta in list(fasta_dict_228.items()) if rfam not in rfams_processed_for_test]
train_superset_fastas = list(set(unprocessed_train_fastas + train_fastas + [v for k,v in rfam_3x_train.items()]))

# CONCAT TRAIN
superset_train = f"{superset_dir}Superset_Train.fa"
if not os.path.exists(superset_train):
    os.system(f"cat {' '.join(train_superset_fastas)} > {superset_train}")

# STATS
# Total train tokens
output = subprocess.check_output(f"seqkit stats {superset_train}", shell=True, text=True)
superset_train_tokens = int(output.split()[12].replace(",", ""))
print(f"## SUPERSET TOTAL TRAIN TOKENS: {superset_train_tokens}")
# Thermo train tokens
superset_train_thermo_tokens = np.sum([len(record.seq) for record in SeqIO.parse(superset_train, "fasta") \
                                if record.id.split("__")[0] in thermo_acc])
print(f"## SUPERSET TRAIN THERMO TOKENS: {superset_train_thermo_tokens}")

## COLLECT SUPERSET FINETUNING-TEST: 5XRFAM Test +  250xRFAM Test
test_superset_fastas = [v for k,v in rfam_3x_test.items()] + test_fastas

# CONCAT TEST
superset_test = f"{superset_dir}Superset_Test.fa"
if not os.path.exists(superset_test):
    os.system(f"cat {' '.join(test_superset_fastas)} > {superset_test}")

# STATS
# Total train tokens
output = subprocess.check_output(f"seqkit stats {superset_test}", shell=True, text=True)
superset_test_tokens = int(output.split()[12].replace(",", ""))
print(f"## SUPERSET TEST TOKENS: {superset_test_tokens}")
# Thermo train tokens
superset_test_thermo_tokens = np.sum([len(record.seq) for record in SeqIO.parse(superset_test, "fasta") \
                                if record.id.split("__")[0] in thermo_acc])
print(f"## SUPERSET TEST THERMO TOKENS: {superset_test_thermo_tokens}")

total_superset_tokens = superset_train_tokens + superset_test_tokens
print(f"* Superset TEST Tokens are {100*superset_test_tokens/total_superset_tokens}% Total")
print(f"* Superset TEST THERMO Tokens are {100*superset_test_thermo_tokens/total_superset_tokens}% Total")
print("# ALL MINI RFAM PROCESSED: ", len(unprocessed_train_fastas) + len(rfams_processed_for_test) == 228)


## SUPERSET TOTAL TRAIN TOKENS: 274214273
## SUPERSET TRAIN THERMO TOKENS: 2605716
## SUPERSET TEST TOKENS: 30805909
## SUPERSET TEST THERMO TOKENS: 949507
* Superset TEST Tokens are 10.099629735320269% Total
* Superset TEST THERMO Tokens are 0.31129317206951246% Total
# ALL MINI RFAM PROCESSED:  True


In [10]:
#### EXTENDED CHECK SUMS ####
## CHECK TEST IDS UNIQUE AND COMPLETE
rfam_3x_ids = [[record.id for record in SeqIO.parse(fasta, "fasta")] for fasta in rfam_3x_test.values()]
rfam_3x_ids = set([s for l in rfam_3x_ids for s in l])
n_supertest_ids = len([record.id for record in SeqIO.parse(superset_test, "fasta")])
print("## SUPER TEST CORRECT: ", len(flat_test_ids) + len(rfam_3x_ids) == n_supertest_ids)

## CHECK TRAIN IDS UNIQUE AND COMPLETE
rfam_3x_ids = [[record.id for record in SeqIO.parse(fasta, "fasta")] for fasta in rfam_3x_train.values()]
rfam_3x_ids = set([s for l in rfam_3x_ids for s in l])
n_supertrain_ids = len(set([record.id for record in SeqIO.parse(superset_train, "fasta")]))
print("## SUPER TRAIN CORRECT: ", len(rfam_3x_ids) + len(minirfam_fasta_len_dict.keys())-len(flat_test_ids) == n_supertrain_ids)

## CHECK TEST-SET IDS NOT DUPLICATED
flat_test_ids = [s for l in test_set_ids for s in l]
counted = Counter(flat_test_ids)
print("# TEST IDs NOT DUPLICATED: ", len([k.split("__")[-1] for k,v in counted.items() if v > 1]) == 0)

# GET NUMBER OF SINGLETS PROCESSED
print("# NUM SINGLETS PROCESSED: ", len([x for x in singlets if x in rfams_processed_for_test])) # N Singlets used for Test

## SUPER TEST CORRECT:  True
## SUPER TRAIN CORRECT:  True
# TEST IDs NOT DUPLICATED:  True
# NUM SINGLETS PROCESSED:  1


In [11]:
#### CREATE FINETUNING SETS: SPLIT THERMOPHILE FASTAS ####
n_records_in_superset_test = len([record.id for record in SeqIO.parse(superset_test, "fasta") if record.id.split("__")[0]])
thermos_in_super_train = [record.id for record in SeqIO.parse(superset_train, "fasta") if record.id.split("__")[0] in thermo_acc]
thermos_in_super_test = [record.id for record in SeqIO.parse(superset_test, "fasta") if record.id.split("__")[0] in thermo_acc]

superset_test_index = list(SeqIO.parse(superset_test, "fasta"))
indices = list(range(0, n_records_in_superset_test))

# Ten percent of testing tokens
tokens_for_thermo_test = int(np.ceil(fraction*superset_test_thermo_tokens))
n_thermo_test_tokens = 0
thermo_seqs_selected = []

# WRITE THERMO TEST: Collect 10% of tokens for seqs not in initial training
thermo_test_fasta = f"{superset_dir}Superset_Thermo_Test.fa"
n_thermo_test = 0
with open(thermo_test_fasta, "w") as f:
    while n_thermo_test_tokens < tokens_for_thermo_test:
        # Select a random sequence from superset-test
        random_index = np.random.choice(indices)
        while random_index in thermo_seqs_selected:
            random_index = np.random.choice(indices)
        thermo_seqs_selected.append(random_index)
        
        # Get seq info
        random_read_id = superset_test_index[random_index].id
        if random_read_id in thermos_in_super_test:
            random_read_seq = superset_test_index[random_index].seq
            # Write to thermo-test
            f.write(f">{random_read_id}\n{random_read_seq}\n")
            n_thermo_test += 1
            n_thermo_test_tokens += len(random_read_seq)
        if len(thermo_seqs_selected) > n_records_in_superset_test:
            break

# WRITE THERMO FINETUNING: use all seqs not reserved for Thermo-Test
indices_not_in_thermo_test = [x for x in indices if x not in thermo_seqs_selected]
finetuning_fasta = f"{superset_dir}Superset_Thermo_Finetune.fa"
n_finetune_tokens = 0
n_finetune = 0
with open(finetuning_fasta, "w") as f:
    for idx in indices_not_in_thermo_test:
        random_read_id = superset_test_index[idx].id
        
        # If thermophile, save for finetuning
        #if random_read_id in thermos_in_super_test:
        if random_read_id.split("__")[0] in thermo_acc:
            random_read_seq = superset_test_index[idx].seq
            # Write to thermo finetuning
            f.write(f">{random_read_id}\n{random_read_seq}\n")
            n_finetune += 1
            n_finetune_tokens += len(random_read_seq)
    
    # Check all of superset test processed
    print(n_thermo_test + n_finetune == len(thermos_in_super_test))
    
    # Add thermos from superset train
    for record in SeqIO.parse(superset_train, "fasta"):
        random_read_id = record.id
        if random_read_id.split("__")[0] in thermo_acc:
            # Write to thermo finetuning
            f.write(f">{random_read_id}\n{record.seq}\n")
            n_finetune += 1
            n_finetune_tokens += len(record.seq)

# CHECK SUM
print(n_finetune_tokens, n_thermo_test_tokens)

True
3458844 96379


In [12]:
#### VALIDATION ####
# CHECK FOR DATA LEAKAGE: Superset_Train.fa and Superset_Test.fa
superset_train_ids = set([record.id for record in SeqIO.parse(superset_train,"fasta")])
superset_test_ids = set([record.id for record in SeqIO.parse(superset_test,"fasta")])
print(f"# Check Superset_Train.fa and Superset_Test.fa overlap: {len(superset_test_ids&superset_train_ids)}")

# CHECK FOR DATA LEAKAGE: Superset_Train.fa and Superset_Thermo_Test.fasta
superset_thermo_test_ids = set([record.id for record in SeqIO.parse(thermo_test_fasta,"fasta")])
print(f"# Check Superset_Train.fa and Superset_Thermo_Test.fasta overlap: {len(superset_thermo_test_ids&superset_train_ids)}")


# Check Superset_Train.fa and Superset_Test.fa overlap: 0
# Check Superset_Train.fa and Superset_Thermo_Test.fasta overlap: 0
